In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
df_mov = pd.read_csv('https://raw.githubusercontent.com/iad34/seminars/master/materials/recsys/movies.dat', sep='::', 
                     engine='python', header=None)

In [4]:
df_mov.columns = ['id', 'name', 'genre']

In [5]:
df_mov.head()

,id,name,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
df_mov.loc[df_mov['name'].str.contains('Star Wars')]

,id,name,genre
257,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
1178,1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War
1192,1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
2559,2628,Star Wars: Episode I - The Phantom Menace (1999),Action|Adventure|Fantasy|Sci-Fi


In [7]:
df_train = pd.read_csv('https://raw.githubusercontent.com/iad34/seminars/master/materials/recsys/train.csv', sep=',', 
                     engine='python')

In [8]:
df_train.head()

,user,item,rating
0,1,3186,0.8
1,1,1270,1.0
2,1,1721,0.8
3,1,1022,1.0
4,1,2340,0.6


In [9]:
n_users = df_train['user'].max()
n_items = df_train['item'].max()

In [11]:
ratings = np.zeros((n_users+1, n_items+1))
ratings[df_train['user'], df_train['item']] = df_train['rating']

In [13]:
# srednaya ocenka filma s id = 260
# 0 - ocenki net
ratings[:, 260][ratings[:,260] != 0].mean()

0.89118406492069346

In [23]:
# srednie ocenki dlia kazhdogo u
v = []
for u in range(n_users+1):
    v.append(ratings[u,:][ratings[u,:] != 0].mean())
v = np.array(v) 
np.array(v).shape

D:\Anaconda3\lib\site-packages\numpy\core\_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
D:\Anaconda3\lib\site-packages\numpy\core\_methods.py:70: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


(6041,)

In [29]:
v

array([        nan,  0.83809524,  0.77475728, ...,  0.7875    ,
        0.75306122,  0.69705882])

In [19]:
ratings[:,:][ratings[:,:] != 0].mean()

0.72338002251309275

In [27]:
# drugoy sposob
mean_ru = ratings.sum(axis=1)
count_ru = (ratings != 0).astype(int).sum(axis=1)
count_ru[count_ru == 0] = 1
mean_ru = mean_ru/count_ru

In [31]:
mean_ru

array([ 0.        ,  0.83809524,  0.77475728, ...,  0.7875    ,
        0.75306122,  0.69705882])

In [33]:
# kolichestvo ocenok dlia filma 260 i 1196
# u(i, j)
np.count_nonzero(ratings[:,260]*ratings[:,1196])

2039

In [35]:
# polz kto ocenili i to i to
# u_ij - ix indexy
i, j = 260, 1196
u_ij = np.nonzero(ratings[:,i]*ratings[:,j])[0]

In [36]:
# poscitat funkcional
top = np.sum((ratings[u_ij, i] - mean_ru[u_ij])*(ratings[u_ij, j] - mean_ru[u_ij]))
botm = np.sqrt(np.sum((ratings[u_ij, i] - mean_ru[u_ij])**2)*np.sum((ratings[u_ij, j] - mean_ru[u_ij])**2))
top/botm

0.80787019060654941

In [42]:
R = ratings - (ratings != 0) * mean_ru[:, np.newaxis]

In [44]:
top2 = np.dot(R.T, R)

In [45]:
botm2 = np.dot(R.T**2, R != 0.0)
botm2 = np.sqrt(botm2*botm2.T)
botm2[botm2==0]=1
sim = top2/botm2

In [46]:
sim[260, 1196]

0.80821987766552028